In [1]:
# Core
import numpy as np
import pandas as pd

# DB
import sqlalchemy
from sqlalchemy import create_engine

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from db_utils import sp2df
from astro_utils import mjd_to_datetime
from rebound_sim import make_sim_planets
from rebound_integrate import integrate_mjds
# from ztf_data_load import db_engine, load_ztf_classes, load_ztf_objects, load_ztf_detections, ztf_detection_add_dir

In [3]:
# Distinct ZTF detection times
dt = sp2df('ZTF.GetDetectionTimes')

In [4]:
dt[0:5]

,DetectionTimeID,MJD
0,1,58270.167662
1,2,58270.168125
2,3,58270.169965
3,4,58270.170417
4,5,58270.171343


In [5]:
df = dt.copy()

In [6]:
# Array of observations times as MJDs
mjds = dt['MJD'].values

In [7]:
# Populate the CalendarDateTime field
df['CalendarDateTime'] = np.array([mjd_to_datetime(mjd) for mjd in mjds])

In [8]:
# Calculate Earth position
sim_epoch = make_sim_planets(epoch=mjds[0])

In [9]:
integrate_mjds(sim_epoch=sim_epoch, mjds=mjds[0:100], save_elements=False, progbar=True)

  0%|          | 0/100 [00:00<?, ?it/s]

0it [00:00, ?it/s]

(array([ 10,   1,   2, 399, 301,   4,   5,   6,   7,   8,   9]),
 array(['Sun', 'Mercury Barycenter', 'Venus Barycenter', 'Earth', 'Moon',
        'Mars Barycenter', 'Jupiter Barycenter', 'Saturn Barycenter',
        'Uranus Barycenter', 'Neptune Barycenter', 'Pluto Barycenter'],
       dtype=object),
 array([[[ 8.40567333e-04,  6.80788190e-03, -9.71296235e-05],
         [ 2.29552201e-01,  2.25720450e-01, -3.19090524e-03],
         [-6.61889512e-01,  2.82219428e-01,  4.19247365e-02],
         ...,
         [ 1.74409862e+01,  9.56013848e+00, -1.90443874e-01],
         [ 2.88108798e+01, -8.13370514e+00, -4.96477568e-01],
         [ 1.12361856e+01, -3.16301897e+01,  1.34453204e-01]],
 
        [[ 8.40564133e-04,  6.80788372e-03, -9.71295462e-05],
         [ 2.29540619e-01,  2.25730413e-01, -3.18902886e-03],
         [-6.61893152e-01,  2.82210738e-01,  4.19248272e-02],
         ...,
         [ 1.74409853e+01,  9.56013999e+00, -1.90443857e-01],
         [ 2.88108802e+01, -8.13370374e+00, -4

In [10]:
# Calculate topos adjustment
dq_topos = 0

In [11]:
df[0:5]

,DetectionTimeID,MJD,CalendarDateTime
0,1,58270.167662,2018-06-01 04:01:25.996783
1,2,58270.168125,2018-06-01 04:02:05.999991
2,3,58270.169965,2018-06-01 04:04:45.001903
3,4,58270.170417,2018-06-01 04:05:24.002897
4,5,58270.171343,2018-06-01 04:06:44.000648
